In [18]:
from time import time
import requests
import pandas as pd
import numpy as np 
import random 
import datetime
import os
import pickle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [19]:
os.chdir('C:/Users/patrick.franco/Documents/GitHub/spread/RAROC/dados')
os.getcwd()

'C:\\Users\\patrick.franco\\Documents\\GitHub\\spread\\RAROC\\dados'

In [20]:
%%time
naoautom = pd.read_excel('naoautomaticas.xlsx', sheet_name='SITE', 
                         converters={'CNPJ': str, 'Município - código': str})

Wall time: 10.4 s


In [21]:
%%time

naoautom['cnpjbasico'] = naoautom['CNPJ'].str[:8]
naoautom['ano']   = naoautom['Data da contratação'].dt.year

Wall time: 19.8 ms


In [22]:
naoautom.dtypes

Cliente                                               object
CNPJ                                                  object
Descrição do projeto                                  object
UF                                                    object
Município                                             object
Município - código                                    object
Número do contrato                                     int64
Data da contratação                           datetime64[ns]
Valor contratado  R$                                 float64
Valor desembolsado R$                                float64
Fonte de recurso (desembolsos)                        object
Custo financeiro                                      object
Juros                                                float64
Prazo - carência (meses)                               int64
Prazo - amortização (meses)                            int64
Modalidade de apoio                                   object
Forma de apoio          

In [23]:
%%time
naoautom.head()

Wall time: 985 µs


,Cliente,CNPJ,Descrição do projeto,UF,Município,Município - código,Número do contrato,Data da contratação,Valor contratado R$,Valor desembolsado R$,...,Subsetor BNDES,Porte do cliente,Natureza do cliente,Instituição Financeira Credenciada,CNPJ da instituição financeira credenciada,Tipo de garantia,Tipo de excepcionalidade,Situação do contrato,cnpjbasico,ano
0,SOL PR CIDADE GAUCHA S.A.,35492397000160,"IMPLANTACAO DE 17 USINAS FOTOVOLTAICAS, LOCALI...",PR,CIDADE GAUCHA,4105607,22203621,2022-12-29,11290000.0,0.0,...,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO,35492397,2022
1,SOL PR CIDADE GAUCHA S.A.,35492397000160,"IMPLANTACAO DE 17 USINAS FOTOVOLTAICAS, LOCALI...",PR,CIDADE GAUCHA,4105607,22203621,2022-12-29,11290000.0,0.0,...,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO,35492397,2022
2,SOL PR LOANDA S.A.,33421628000110,"IMPLANTACAO DE 17 USINAS FOTOVOLTAICAS, LOCALI...",PR,LOANDA,4113502,22203621,2022-12-29,8627000.0,0.0,...,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO,33421628,2022
3,SOL PR LOANDA S.A.,33421628000110,"IMPLANTACAO DE 17 USINAS FOTOVOLTAICAS, LOCALI...",PR,LOANDA,4113502,22203621,2022-12-29,8627000.0,0.0,...,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO,33421628,2022
4,SOL PR NOVA ESPERANCA S.A.,34061082000104,"IMPLANTACAO DE 17 USINAS FOTOVOLTAICAS, LOCALI...",PR,ALTO PARANA,4100608,22203621,2022-12-29,11276000.0,0.0,...,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL / PESSOAL,----------,ATIVO,34061082,2022


In [24]:
#naoautom.drop_duplicates(subset=['CNPJ'])

In [26]:
%%time
painel = naoautom.groupby(['cnpjbasico', 'ano']).agg({'Número do contrato':'count', 
                                                 'Valor contratado  R$':'sum',
                                                 'Valor desembolsado R$':'sum',
                                                 'Juros':'mean',
                                                 'Prazo - carência (meses)':'mean',
                                                 'Prazo - amortização (meses)':'mean'}).reset_index()


Wall time: 43.9 ms


In [34]:
painel[['cnpjbasico', 'ano']].groupby('cnpjbasico').count().sort_values(by='ano',ascending=False)

,ano
cnpjbasico,
16404287,19
77752293,17
76093731,16
75904383,16
89637490,14
...,...
10955061,1
10960725,1
10962697,1


In [43]:
painel[painel['cnpjbasico']=='16404287'].head(25).sort_values(by='ano',ascending=False)

,cnpjbasico,ano,Número do contrato,Valor contratado R$,Valor desembolsado R$,Juros,Prazo - carência (meses),Prazo - amortização (meses)
3405,16404287,2022,2,2.313647e+09,0.000000e+00,1.700000,54.000000,156.000000
3404,16404287,2021,1,3.100000e+08,2.430000e+08,1.560000,24.000000,156.000000
3403,16404287,2020,1,4.796530e+08,4.317740e+08,1.850000,12.000000,228.000000
3402,16404287,2019,1,7.061270e+08,6.000000e+08,1.770000,84.000000,96.000000
3401,16404287,2018,9,2.607030e+08,2.143100e+08,1.493333,18.000000,73.333333
3400,16404287,2017,3,6.905950e+08,4.641081e+08,2.560000,62.000000,36.000000
3399,16404287,2016,13,2.648035e+09,2.132051e+09,2.113846,22.153846,69.230769
3398,16404287,2015,9,3.129125e+08,1.934498e+08,4.454444,36.000000,49.333333
3397,16404287,2014,10,4.191380e+08,4.117625e+08,3.652000,19.800000,62.400000
3396,16404287,2013,15,8.917175e+08,6.578243e+08,2.608000,41.400000,47.400000


In [1]:
%%time
import bz2
import pickle
import _pickle as cPickle
import pyreadstat

Wall time: 2.83 s


In [2]:
%%time
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

df_fls = decompress_pickle('C:/Dados Socios/unificado/filiais.pbz2')[
    ['cnpjbasico','sit_cadastral','data_sit','mot_cadastral','data_inicio']
    ].drop_duplicates(subset=['cnpjbasico','data_sit','mot_cadastral','data_inicio'])

Wall time: 15min 26s


In [4]:
%%time
df_fls.dtypes

Wall time: 0 ns


cnpjbasico       object
sit_cadastral     int64
data_sit          int64
mot_cadastral     int64
data_inicio       int64
dtype: object

In [5]:
%%time
df_fls['ano_sit']    = df_fls['data_sit'].astype(str).str[:4]
df_fls['ano_inicio'] = df_fls['data_inicio'].astype(str).str[:4]

Wall time: 1min 46s


In [7]:
%%time
df_fls.shape

Wall time: 337 µs


(42381618, 7)

In [11]:
%%time
df_fls['ano_sit']    = pd.to_numeric(df_fls['ano_sit'])
df_fls['ano_inicio'] = pd.to_numeric(df_fls['ano_inicio'])

Wall time: 53.3 s


In [12]:
%%time
df_fls.head()

Wall time: 609 µs


,cnpjbasico,sit_cadastral,data_sit,mot_cadastral,data_inicio,ano_sit,ano_inicio
0,07396865,8,20170210,1,20050518,2017,2005
1,64904295,8,20161110,1,20050429,2016,2005
2,76016369,3,20060203,21,19851212,2006,1985
3,52302726,4,20210406,63,19830223,2021,1983
4,07396923,8,20140115,1,20050516,2014,2005


In [67]:
%%time
df_fls.drop_duplicates(subset=['cnpjbasico','ano_sit','mot_cadastral'],keep='first',inplace=True)

Wall time: 48.6 s


In [71]:
%%time
df_fls[df_fls['cnpjbasico']=='16404287'].head(25).sort_values(by=['ano_sit','sit_cadastral','mot_cadastral'],ascending=False)

Wall time: 2.7 s


,cnpjbasico,sit_cadastral,data_sit,mot_cadastral,data_inicio,ano_sit,ano_inicio
4039897,16404287,8,20220121,1,20190605,2022,2019
353226,16404287,2,20220106,0,20220106,2022,2022
255727,16404287,8,20211227,1,20191030,2021,2019
4747395,16404287,2,20210416,0,20210416,2021,2021
325174,16404287,8,20201106,1,20040726,2020,2004
4586542,16404287,2,20201127,0,20201127,2020,2020
3254262,16404287,8,20190227,1,20150204,2019,2015
3905965,16404287,2,20190102,0,20190102,2019,2019
1646753,16404287,8,20181106,1,20041013,2018,2004
3785443,16404287,2,20180717,0,20180717,2018,2018


In [59]:
%%time
df_fls.shape

Wall time: 0 ns


(41514484, 7)

In [60]:
%%time
df_fls.head()

Wall time: 0 ns


,cnpjbasico,sit_cadastral,data_sit,mot_cadastral,data_inicio,ano_sit,ano_inicio
0,07396865,8,20170210,1,20050518,2017,2005
1,64904295,8,20161110,1,20050429,2016,2005
2,76016369,3,20060203,21,19851212,2006,1985
3,52302726,4,20210406,63,19830223,2021,1983
4,07396923,8,20140115,1,20050516,2014,2005


In [61]:
%%time
#del df_fls

Wall time: 0 ns


In [62]:
%%time
bndes = pd.merge( painel, df_fls, left_on=['cnpjbasico','ano'], right_on=['cnpjbasico','ano_sit'], how='inner')

Wall time: 1min 21s


Wall time: 5.93 s


,cnpjbasico,sit_cadastral,data_sit,mot_cadastral,data_inicio,ano_sit,ano_inicio
353226,16404287,2,20220106,0,20220106,2022,2022
4039897,16404287,8,20220121,1,20190605,2022,2019
255727,16404287,8,20211227,1,20191030,2021,2019
4747395,16404287,2,20210416,0,20210416,2021,2021
325174,16404287,8,20201106,1,20040726,2020,2004
4586542,16404287,2,20201127,0,20201127,2020,2020
3254262,16404287,8,20190227,1,20150204,2019,2015
3905965,16404287,2,20190102,0,20190102,2019,2019
1646753,16404287,8,20181106,1,20041013,2018,2004
3785443,16404287,2,20180717,0,20180717,2018,2018


In [66]:
bndes[bndes['cnpjbasico']=='16404287'].head(25).sort_values(by='ano',ascending=False)

,cnpjbasico,ano,Número do contrato,Valor contratado R$,Valor desembolsado R$,Juros,Prazo - carência (meses),Prazo - amortização (meses),sit_cadastral,data_sit,mot_cadastral,data_inicio,ano_sit,ano_inicio
793,16404287,2020,1,4.796530e+08,4.317740e+08,1.850000,12.000000,228.000000,8,20201106,1,20040726,2020,2004
792,16404287,2019,1,7.061270e+08,6.000000e+08,1.770000,84.000000,96.000000,2,20190102,0,20190102,2019,2019
791,16404287,2019,1,7.061270e+08,6.000000e+08,1.770000,84.000000,96.000000,8,20190227,1,20150204,2019,2015
790,16404287,2018,9,2.607030e+08,2.143100e+08,1.493333,18.000000,73.333333,8,20181106,1,20041013,2018,2004
789,16404287,2018,9,2.607030e+08,2.143100e+08,1.493333,18.000000,73.333333,2,20180717,0,20180717,2018,2018
788,16404287,2017,3,6.905950e+08,4.641081e+08,2.560000,62.000000,36.000000,2,20170315,0,20170315,2017,2017
787,16404287,2017,3,6.905950e+08,4.641081e+08,2.560000,62.000000,36.000000,8,20171002,1,20040726,2017,2004
786,16404287,2016,13,2.648035e+09,2.132051e+09,2.113846,22.153846,69.230769,8,20160706,1,20070914,2016,2007
785,16404287,2016,13,2.648035e+09,2.132051e+09,2.113846,22.153846,69.230769,2,20161019,0,20161019,2016,2016
784,16404287,2015,9,3.129125e+08,1.934498e+08,4.454444,36.000000,49.333333,8,20151001,1,20041021,2015,2004
